# Clustering and differential expression analysis using scanpy

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import json
import os
import matplotlib.pyplot as plt
from scipy.stats import rankdata

In [ ]:
dataset_name = 'E175'
data_out_path = './data/{}/scanpy/'.format(dataset_name)
default_path = '{}default/'.format(data_out_path)
celltypist_path = '{}celltypist/'.format(data_out_path)
nclusters_celltypist_path = './data/{}/nclusters.json'.format(dataset_name)

top_number_of_markers = 500
min_cluster_size = 0

with open(nclusters_celltypist_path) as f:
    nclusters_celltypist = json.load(f)['nclusters']

nclusters_threshold = 1/10
min_ncluster_celltypist = round(nclusters_celltypist - nclusters_threshold*nclusters_celltypist)
max_ncluster_celltypist = round(nclusters_celltypist + nclusters_threshold*nclusters_celltypist)

if not os.path.exists(default_path):
    os.makedirs(default_path)

if not os.path.exists(celltypist_path):
    os.makedirs(celltypist_path)

## Data loading and preparation

Dataset loading

In [ ]:
adata = sc.read_csv(f'./data/{dataset_name}/CorticalCells_GSM2861514_E175_cleaned.csv', first_column_names=None)
adata = adata.T
adata.var_names_make_unique()
adata

Studying feature variance

In [ ]:
matrix = adata.X
neg_variances = np.sort(-np.var(matrix, axis=0))
sorted_log_variances = [np.log(-i) for i in neg_variances.T][:2000]
plt.scatter([i for i in range(len(sorted_log_variances))], sorted_log_variances)

Data normalization

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

Feature selection

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=500, # chosen based on elbow plot above
    flavor='seurat_v3'
)
sc.pl.highly_variable_genes(adata)

Feature scaling

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10) # default=None, 10 in the tutorial on pbmc

## Clustering

Plot pca explained variance ratio to choose number of components

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps=30, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata, log=True)

### Clustering with default parameters

Perform the clustering

In [ ]:
sc.pp.neighbors(
    adata,
    n_pcs=10 # chosen based on elbow plot above
)
sc.tl.leiden(adata)

Visualize the clustering in the PCA space

In [ ]:
sc.pl.pca(
    adata,
    color=['leiden'],
    size=20
)

Save the clustering results

In [ ]:
default_markers_df = pd.DataFrame(adata.obs['leiden'])
default_markers_df = default_markers_df.rename(columns={'leiden': 'cluster'})
default_markers_df.index.name = 'cell'
default_markers_df['cluster'] = default_markers_df['cluster'].astype(int) + 1
default_markers_df.to_csv(f'{default_path}/clustering_labels.csv', index=True)
num_clusters_default = len(default_markers_df['cluster'].unique())

## Differential expression on default clusters

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon') # default='t-test', wilcoxon suggested in the tutorial

Save markers

In [ ]:
default_markers_df = pd.DataFrame(columns=['gene', 'cluster', 'rank'])
for i in range(num_clusters_default):
    indices = np.argsort(adata.uns['rank_genes_groups']['pvals_adj'][str(i)])
    genes = adata.uns['rank_genes_groups']['names'][str(i)][indices][:top_number_of_markers]
    ranks = rankdata(adata.uns['rank_genes_groups']['pvals_adj'][str(i)][indices][:top_number_of_markers], method="ordinal")
    temp_df = pd.DataFrame(
        np.column_stack((genes, [i+1 for _ in range(top_number_of_markers)], ranks)),
        columns=['gene', 'cluster', 'rank']
    )
    default_markers_df = pd.concat([default_markers_df, temp_df], ignore_index=True)

default_markers_df['cluster'] = default_markers_df['cluster'].astype(int)
default_markers_df['rank'] = default_markers_df['rank'].astype(int)
default_markers_df.to_csv(f'{default_path}/markers.csv', index=False)

### Clustering tuning resolution according to celltypist

In [ ]:
print("Nummber of clusters to find: {}".format(nclusters_celltypist))

Perform the clustering

In [ ]:
sc.pp.neighbors(
    adata,
    n_pcs=10 # chosen based on elbow plot above
)

max_resolution = 3
min_resolution = 0

while True:
    resolution = (max_resolution + min_resolution)/2
    print("Trying resolution: {}".format(resolution))
    sc.tl.leiden(adata, key_added="leiden", resolution=resolution)
    num_clusters = adata.obs.leiden.values.categories.nunique()
    print("Number of clusters found: {}".format(num_clusters))

    if num_clusters >= min_ncluster_celltypist and num_clusters <= max_ncluster_celltypist:
        break
    elif num_clusters <  min_ncluster_celltypist:
        min_resolution = resolution
    else:
        max_resolution = resolution

Visualize the clustering in the PCA space

In [ ]:
sc.pl.pca(
    adata,
    color=['leiden'],
    size=20
)

Save the clustering results

In [ ]:
celltypist_markers_df = pd.DataFrame(adata.obs['leiden'])
celltypist_markers_df = celltypist_markers_df.rename(columns={'leiden': 'cluster'})
celltypist_markers_df.index.name = 'cell'
celltypist_markers_df['cluster'] = celltypist_markers_df['cluster'].astype(int) + 1
celltypist_markers_df.to_csv(f'{celltypist_path}/clustering_labels.csv', index=True)
celltypist_num_clusters = len(celltypist_markers_df['cluster'].unique())

## Differential expression on clusters tuned according to celltypist

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon') # default='t-test', wilcoxon suggested in the tutorial

Save markers

In [ ]:
celltypist_markers_df = pd.DataFrame(columns=['gene', 'cluster', 'rank'])
for i in range(celltypist_num_clusters):
    indices = np.argsort(adata.uns['rank_genes_groups']['pvals_adj'][str(i)])
    genes = adata.uns['rank_genes_groups']['names'][str(i)][indices][:top_number_of_markers]
    ranks = rankdata(adata.uns['rank_genes_groups']['pvals_adj'][str(i)][indices][:top_number_of_markers], method="ordinal")
    temp_df = pd.DataFrame(
        np.column_stack((genes, [i+1 for _ in range(top_number_of_markers)], ranks)),
        columns=['gene', 'cluster', 'rank']
    )
    celltypist_markers_df = pd.concat([celltypist_markers_df, temp_df], ignore_index=True)

celltypist_markers_df['cluster'] = celltypist_markers_df['cluster'].astype(int)
celltypist_markers_df['rank'] = celltypist_markers_df['rank'].astype(int)
celltypist_markers_df.to_csv(f'{celltypist_path}/markers.csv', index=False)